In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#Importing Data 
tourn_csv_data = pd.read_csv('data\\tournaments.csv')[['id','leagueid', 'slug']]
tourn_json_data = pd.read_json('data\\tournaments.jsonl', lines = True).drop(['id', 'leagueId'], axis = 1)
tourn_data = tourn_csv_data.merge(tourn_json_data, on = 'slug')
teams_data = pd.read_csv('data\\teams.csv').set_index('team_id')
match_data = pd.read_csv("data\\match_history.csv").set_index('egameid')
league_data = pd.read_csv("data\\leagues.csv").rename({'name': 'Region', 'id': 'leagueid'}, axis = 1)
# gold_df = pd.read_csv('data\\gold_differences.csv').set_index('platformgameid')

region_tiers = {'t1': {'regions': ['LPL', 'LCK', 'Worlds', 'MSI'],
                        'start_elo': 1500},
                't2': {'regions': ['LCS', 'LEC'],
                        'start_elo': 1250},
                't3': {'regions' : [],
                        'start_elo': 1000}}

In [3]:
display(tourn_data.head())

,id,leagueid,slug,name,sport,startDate,endDate,stages
0,110733838935136200,109518549825754242,nacl_qualifiers_2_summer_2023,#2 Summer 2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,110428723804419399,105266108767593290,gll_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,110424377524465827,107407335299756365,elite_series_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,110349992504762921,105266074488398661,superliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,110310652404523974,105266088231437431,ultraliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-13,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [4]:
input = 'LCK, LPL, LCS, LEC, Worlds, MSI, LJL, LCL'
region_list = [region.strip() for region in input.lower().split(',')]
print(region_list)

['lck', 'lpl', 'lcs', 'lec', 'worlds', 'msi', 'ljl', 'lcl']


In [5]:
#Filter leagues to user input
sel_league_df = league_data.loc[league_data['slug'].isin(region_list)] if len(region_list) > 0 else league_data
display(sel_league_df)

,leagueid,Region,slug,sport,image,lightimage,darkimage,region,priority,displaypriority,tournaments
0,98767991299243165,LCS,lcs,lol,http://static.lolesports.com/leagues/LCSNew-01...,http://static.lolesports.com/leagues/LCSNew-01...,http://static.lolesports.com/leagues/LCSNew-01...,NORTH AMERICA,1,"{position=0, status=selected}","[{id=110303581083678395}, {id=1095170900666056..."
5,98767991310872058,LCK,lck,lol,http://static.lolesports.com/leagues/lck-color...,http://static.lolesports.com/leagues/lck-color...,http://static.lolesports.com/leagues/lck-color...,KOREA,4,"{position=1, status=not_selected}","[{id=110909366079472439}, {id=1103715512775087..."
6,98767991355908944,LCL,lcl,lol,http://static.lolesports.com/leagues/159301688...,http://static.lolesports.com/leagues/159301688...,http://static.lolesports.com/leagues/159301688...,COMMONWEALTH OF INDEPENDENT STATES,209,"{position=2, status=not_selected}","[{id=107417471555810057}, {id=1062697697242688..."
8,98767991302996019,LEC,lec,lol,http://static.lolesports.com/leagues/159251618...,http://static.lolesports.com/leagues/159251618...,http://static.lolesports.com/leagues/159251618...,EMEA,3,"{position=4, status=not_selected}","[{id=110848560874526298}, {id=1104293326886042..."
11,98767991314006698,LPL,lpl,lol,http://static.lolesports.com/leagues/159251611...,http://static.lolesports.com/leagues/159251611...,http://static.lolesports.com/leagues/159251611...,CHINA,201,"{position=7, status=not_selected}","[{id=110825936250664572}, {id=1104288487665643..."
15,98767991325878492,MSI,msi,lol,http://static.lolesports.com/leagues/159259463...,http://static.lolesports.com/leagues/159259463...,http://static.lolesports.com/leagues/159259463...,INTERNATIONAL,301,"{position=11, status=not_selected}","[{id=110198981276611770}, {id=1079052849834701..."
16,98767975604431411,Worlds,worlds,lol,http://static.lolesports.com/leagues/159259461...,http://static.lolesports.com/leagues/159259461...,http://static.lolesports.com/leagues/159259461...,INTERNATIONAL,300,"{position=12, status=not_selected}","[{id=108998961191900167}, {id=1069262823330895..."


In [6]:
#Fixing game data, game time/duration
match_data['event_time'] = pd.to_datetime(match_data['event_time'])
match_data['game_time'] /= 60
match_data['game_time'] = match_data['game_time'].apply(lambda x: f"{str(x)[:2]}:{int((float(str(x)[2:])*(6/100))):02}")
match_data['event_time'] = pd.to_datetime(match_data['event_time'])
# match_data = match_data.loc[match_data['event_time'] > ('2023')]
match_data['event_time'] = match_data['event_time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))


sel_tourn_df = sel_league_df[['leagueid', 'Region']].merge(tourn_data, how='inner', on = 'leagueid')

display(tourn_data.head())
display(match_data.head())
display(sel_tourn_df.head())
display(sel_league_df)

,id,leagueid,slug,name,sport,startDate,endDate,stages
0,110733838935136200,109518549825754242,nacl_qualifiers_2_summer_2023,#2 Summer 2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,110428723804419399,105266108767593290,gll_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,110424377524465827,107407335299756365,elite_series_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,110349992504762921,105266074488398661,superliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,110310652404523974,105266088231437431,ultraliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-13,"[{'name': 'Regular Season', 'type': None, 'slu..."


,gameid,event_time,game_time,blue_team_id,red_team_id,winning_side,winning_team
egameid,,,,,,,
101978171977781193,ESPORTSTMNT01:1120735,2019-05-06 12:27,24:48,98767991902391669,101978171843206569,100,98767991902391669
102181142520557227,ESPORTSTMNT01:1132954,2019-06-12 13:19,23:19,99871276356460471,100725845022060229,200,100725845022060229
101978171965329317,ESPORTSTMNT01:1121419,2019-05-13 15:25,26:32,100783238110621058,98767991853197861,200,98767991853197861
102147202193408304,ESPORTSTMNT01:1161821,2019-07-26 01:10,26:26,99566405123587075,99566405132177721,100,99566405123587075
101978171979157453,ESPORTSTMNT01:1110177,2019-05-01 11:45,26:13,100783238110621058,99566408358330419,100,100783238110621058


,leagueid,Region,id,slug,name,sport,startDate,endDate,stages
0,98767991299243165,LCS,105658534671026792,lcs_summer_2021,LCS Summer 2021,lol,2021-06-04,2021-08-30,"[{'name': 'Regular Season', 'type': None, 'slu..."
1,98767991299243165,LCS,108206581962155974,lcs_summer_2022,LCS Summer 2022,lol,2022-06-16,2022-09-13,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,98767991299243165,LCS,107458335260330212,lcs_lock_in_2022,LCS Lock In 2022,lol,2022-01-14,2022-01-31,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
3,98767991299243165,LCS,105522217230238828,lcs_lock_in_2021,LCS Lock In 2021,lol,2021-01-15,2021-01-31,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
4,98767991299243165,LCS,110303581083678395,lcs_summer_2023,Summer 2023,lol,2023-05-31,2023-08-22,"[{'name': 'Regular Season', 'type': None, 'slu..."


,leagueid,Region,slug,sport,image,lightimage,darkimage,region,priority,displaypriority,tournaments
0,98767991299243165,LCS,lcs,lol,http://static.lolesports.com/leagues/LCSNew-01...,http://static.lolesports.com/leagues/LCSNew-01...,http://static.lolesports.com/leagues/LCSNew-01...,NORTH AMERICA,1,"{position=0, status=selected}","[{id=110303581083678395}, {id=1095170900666056..."
5,98767991310872058,LCK,lck,lol,http://static.lolesports.com/leagues/lck-color...,http://static.lolesports.com/leagues/lck-color...,http://static.lolesports.com/leagues/lck-color...,KOREA,4,"{position=1, status=not_selected}","[{id=110909366079472439}, {id=1103715512775087..."
6,98767991355908944,LCL,lcl,lol,http://static.lolesports.com/leagues/159301688...,http://static.lolesports.com/leagues/159301688...,http://static.lolesports.com/leagues/159301688...,COMMONWEALTH OF INDEPENDENT STATES,209,"{position=2, status=not_selected}","[{id=107417471555810057}, {id=1062697697242688..."
8,98767991302996019,LEC,lec,lol,http://static.lolesports.com/leagues/159251618...,http://static.lolesports.com/leagues/159251618...,http://static.lolesports.com/leagues/159251618...,EMEA,3,"{position=4, status=not_selected}","[{id=110848560874526298}, {id=1104293326886042..."
11,98767991314006698,LPL,lpl,lol,http://static.lolesports.com/leagues/159251611...,http://static.lolesports.com/leagues/159251611...,http://static.lolesports.com/leagues/159251611...,CHINA,201,"{position=7, status=not_selected}","[{id=110825936250664572}, {id=1104288487665643..."
15,98767991325878492,MSI,msi,lol,http://static.lolesports.com/leagues/159259463...,http://static.lolesports.com/leagues/159259463...,http://static.lolesports.com/leagues/159259463...,INTERNATIONAL,301,"{position=11, status=not_selected}","[{id=110198981276611770}, {id=1079052849834701..."
16,98767975604431411,Worlds,worlds,lol,http://static.lolesports.com/leagues/159259461...,http://static.lolesports.com/leagues/159259461...,http://static.lolesports.com/leagues/159259461...,INTERNATIONAL,300,"{position=12, status=not_selected}","[{id=108998961191900167}, {id=1069262823330895..."


In [7]:
mh_df = pd.DataFrame(columns = ['game_id', 'tournament_id', 'league_id', 'blue_team_id', 'red_team_id', 'winning_team_id'])
for tournament in sel_tourn_df.itertuples():
    # print([len(stage['sections']) for stage in tournament.stages])
    for stage in tournament.stages:
        #Standardize Stage names
        stage['name'] = ' '.join([word.capitalize() for word in stage['name'].replace(' ', '_').split('_')])
        # print(stage['name'])
        for match in stage['sections'][0]['matches']:
            for game in match['games']:
                if(game['state'] == 'completed'):
                    #Remove region information from the tournament name in order to standardize tournament names
                    split = ' '.join([word if word.lower() not in league_data['Region'].apply(lambda x: x.lower()).values else '' for word in ''.join([i for i in tournament.name if not i.isdigit()]).replace('_', ' ').split(' ')])
                    year = ''.join([i for i in tournament.name if i.isdigit()])

                    mh_df = mh_df.append({'game_id': int(game['id']), 
                                          'game_num': int(game['number']),
                                        'tournament_id': int(tournament.id),
                                        'tournament': f"{tournament.Region} {split}{stage['name']} {year}",
                                        'stage': stage['name'],
                                        'region': tournament.Region,
                                        'league_id': int(tournament.leagueid),
                                        'blue_team_id': int(game['teams'][0]['id']), 
                                        'red_team_id': int(game['teams'][1]['id']),
                                        'winning_side': 100 if game['teams'][0]['result']['outcome'] == 'win' else 200}, ignore_index=True)
                                        # 'winning_team_id': int(game['teams'][0]['id']) if game['teams'][0]['result']['outcome'] == 'win' else int(game['teams'][1]['id'])}, ignore_index=True)


In [8]:
team_rep = 'acronym'

mh_df = mh_df.merge(teams_data[f"{team_rep}"], left_on = 'blue_team_id', right_index = True, how = 'inner').rename({f"{team_rep}":'blue_side'}, axis = 1)
mh_df = mh_df.merge(teams_data[f"{team_rep}"], left_on = 'red_team_id', right_index = True, how = 'inner').rename({f"{team_rep}":'red_side'}, axis = 1)
# mh_df = mh_df.merge(teams_df[f"{team_rep}"], left_on = 'winning_team_id', right_index = True, how = 'inner').rename({f"{team_rep}":'winner'}, axis = 1)
mh_df = mh_df.merge(match_data[['gameid', 'event_time', 'game_time']], left_on = 'game_id', right_index = True, how = 'inner').rename({f"gameid":'platform_gameid'}, axis = 1)
mh_df[['winning_side', 'game_num']] = mh_df[['winning_side', 'game_num']].astype(int)

mh_df = mh_df.drop_duplicates('platform_gameid').sort_values(by = 'event_time', ascending = True)
teams = set(sorted(set(mh_df['blue_team_id'].values)) +  sorted(set(mh_df['red_team_id'].values)))

In [9]:
def get_start_elo(region):
    for tier, data in region_tiers.items():
        if region in data['regions']:
            return data['start_elo']
    return region_tiers['t3']['start_elo']

def find_home_region(team):
    region_counts = mh_df.loc[(mh_df['blue_team_id'] == team.name) | (mh_df['red_team_id'] == team.name), 'region'].value_counts().index
    if len(region_counts) > 1:
        if region_counts[0] not in ('MSI', 'Worlds'):
            return region_counts[0]
        elif region_counts[1] not in ('MSI', 'Worlds'):
            try:
                return region_counts[1]
            except:
                return 'Wildcard'
        else:
            try:
                return region_counts[2]
            except:
                return 'Wildcard'
    else:
        return region_counts[0] if region_counts[0] not in ('MSI', 'Worlds') else 'Wildcard'

elo_df = teams_data.loc[teams]
elo_df['region'] = elo_df[[f'{team_rep}']].apply(lambda x: find_home_region(x), axis = 1)
elo_df['elo'] = elo_df['region'].apply(lambda x: get_start_elo(x))
elo_df = elo_df.drop_duplicates().sort_index()

In [10]:
elo_df.head()

,name,acronym,slug,region,elo
team_id,,,,,
98767991853197861,T1,T1,t1,LCK,1500
98767991860392497,TSM,TSM,tsm,LCS,1250
98767991866488695,Fnatic,FNC,fnatic,LEC,1250
98767991871709842,Gambit Esports,GMB,gambit-esports,LCL,1000
98767991877340524,Cloud9,C9,cloud9,LCS,1250


In [11]:
display(mh_df[['platform_gameid', 'game_num', 'game_id', 'region', 'tournament', 'stage', 'event_time', 'game_time', 'blue_side', 'red_side', 'winning_side']].loc[mh_df['blue_side'].str.contains('RNG') | mh_df['red_side'].str.contains('RNG')])

print(teams)

,platform_gameid,game_num,game_id,region,tournament,stage,event_time,game_time,blue_side,red_side,winning_side
4625,ESPORTSTMNT06:1720293,1,106032946468612214,MSI,MSI Round 1 2021,Round 1,2021-05-06 15:38,24:12,RNG,PGG,100
4628,ESPORTSTMNT06:1720350,1,106032946468677762,MSI,MSI Round 1 2021,Round 1,2021-05-07 13:36,24:40,PGG,RNG,200
4624,ESPORTSTMNT06:1720352,1,106032946468612212,MSI,MSI Round 1 2021,Round 1,2021-05-07 14:26,23:56,UOL,RNG,200
4627,ESPORTSTMNT06:1720393,1,106032946468677760,MSI,MSI Round 1 2021,Round 1,2021-05-08 13:36,25:01,RNG,UOL,100
4630,ESPORTSTMNT06:1720467,1,106132515649202481,MSI,MSI Round 1 2021,Round 1,2021-05-09 13:41,28:30,UOL,RNG,200
4631,ESPORTSTMNT06:1720480,1,106132515651430707,MSI,MSI Round 1 2021,Round 1,2021-05-09 15:29,31:34,RNG,PGG,100
4633,ESPORTSTMNT06:1720486,1,106132515651496247,MSI,MSI Round 1 2021,Round 1,2021-05-09 16:13,19:38,RNG,UOL,100
4634,ESPORTSTMNT06:1720497,1,106132515651496249,MSI,MSI Round 1 2021,Round 1,2021-05-09 18:00,22:36,PGG,RNG,200
4638,ESPORTSTMNT06:1720675,1,106032946468677822,MSI,MSI Round 2 2021,Round 2,2021-05-14 13:40,27:45,DK,RNG,200
4641,ESPORTSTMNT06:1720687,1,106032946468677828,MSI,MSI Round 2 2021,Round 2,2021-05-14 16:39,29:43,RNG,C9,100


{99566404853854212, 109637393694097670, 106421708401759879, 103963715924353674, 98926509883054987, 99566404846951820, 99566404848691211, 99566406064558732, 99566408221961358, 100285330168091787, 100725845018863243, 98767991871709842, 99566404854685458, 98767991882270868, 104367068120825486, 98767991940181526, 104016416551172375, 101422616509070746, 100205573495116443, 99566408217116828, 105397404796640412, 99566404581868574, 103963753080578719, 105505619546859895, 99566406334639907, 99294153828264740, 98767991853197861, 103535282124208038, 106972778172351142, 101978171843206569, 98767991892579754, 99566404585387054, 98767991860392497, 106269804045322803, 99871276356460471, 105709099258505657, 106806773698472633, 99322214695067838, 99566404855553726, 103461966951059521, 99322214689444546, 99566404853058754, 100725845022060229, 99322214684939974, 102747101562758215, 99566408562477641, 98767991954244555, 99566404850008779, 102747101565183056, 99294153824386385, 105663527959374033, 9956640

In [12]:
display(mh_df[['platform_gameid', 'game_num', 'game_id', 'region', 'tournament', 'stage', 'event_time', 'game_time', 'blue_side', 'red_side', 'winning_side']].drop_duplicates(['region', 'stage']).sort_values('region'))


,platform_gameid,game_num,game_id,region,tournament,stage,event_time,game_time,blue_side,red_side,winning_side
1919,ESPORTSTMNT01:1302595,1,103540364360693634,LCK,LCK Spring SplitRegular Season 2020,Regular Season,2020-02-05 08:53,40:41,T1,DK,200
2096,ESPORTSTMNT03:1342175,1,103540364380027039,LCK,LCK Spring SplitPlayoffs 2020,Playoffs,2020-04-18 09:02,39:30,KT,DK,200
2111,ESPORTSTMNT03:1353399,1,104072420722551893,LCK,LCK Spring SplitPromotion Series 2020,Promotion Series,2020-04-28 08:51,33:29,NS,LSB,100
1863,ESPORTSTMNT03:1433759,1,104174613355356638,LCK,LCK Summer Regional Qualifier 2020,Regional Qualifier,2020-09-07 08:42,36:02,KDF,KT,100
2137,ESPORTSTMNT01:2240116,1,106840590963537535,LCK,LCK Regional Finals Regional Finals 2021,Regional Finals,2021-08-31 08:42,37:32,HLE,LSB,200
3262,ESPORTSTMNT06:1690454,1,105788589450719197,LCL,LCL Spring Playoffs 2021,Playoffs,2021-04-03 13:57,38:19,UOL,GMB,100
3228,ESPORTSTMNT06:1570357,1,105663539175401742,LCL,LCL Spring Regular Season 2021,Regular Season,2021-02-13 13:50,35:27,UOL,GMB,100
457,ESPORTSTMNT03:1673715,1,105522217233187995,LCS,LCS Lock In Knockouts 2021,Knockouts,2021-01-23 21:26,24:27,100,IMT,100
1054,ESPORTSTMNT02:1270555,1,103462440145619666,LCS,LCS Spring Regular Season 2020,Regular Season,2020-01-25 22:31,24:34,C9,TL,100
1131,ESPORTSTMNT02:1333569,1,103462440151845733,LCS,LCS Spring Playoffs 2020,Playoffs,2020-04-04 20:32,29:35,EG,FLY,100


In [13]:
def elo_change(blue_elo, red_elo, winner, region, stage, year):
    stage_values = {'Regionals': {
                        'Promotion Series': 15,
                        'Groups': 20,
                        'Regular Season': 20,
                        'Knockouts': 30,
                        'Playoffs': 30,
                        'Regional Qualifier': 30,
                        'Regional Finals': 35,
                    },
                    'MSI': {
                        'Play In Knockouts': 35,
                        'Round 1': 35,
                        'Round 2': 40,
                        'Knockouts': 45,
                        'Bracket Stage': 45,
                    },
                    'Worlds': {
                        'Play In Groups': 45,
                        'Play In Knockouts': 50,
                        'Groups': 55,
                        'Knockouts': 65,
                        'Playoffs': 65,
                    }
    }
    win_mul = 0.5**(2023 - int(year))
    loss_mul = 0.5**(2023 - int(year))
    if 'MSI' in region:
        match_type = 'MSI'
        if stage in ['Knockouts', 'Bracket Stage']:
            loss_mul *= 0.25
        else:
            loss_mul *= 0.5
    elif 'Worlds' in region:
        match_type = 'Worlds'
        if stage in ['Knockouts', 'Playoffs']:
            loss_mul *= 0
        else:
            loss_mul *= 0.25
    else:
        match_type = 'Regionals'
        if stage in ['Knockouts', 'Playoffs', 'Regional Finals', 'Regional Qualifier']:
            loss_mul *= 0.5
        else:
            loss_mul *= 1
    try:
        base_value = stage_values[match_type][stage]
    except: 
        base_value = 20
        print(region, stage)

    #1 for win, 0 for loss.
    blue_exp_res = 1 / (10**((-(blue_elo - red_elo))/600) + 1)
    red_exp_res = 1 / (10**((-(red_elo - blue_elo))/600) + 1)


    if winner == 100:
        blue_delta = base_value * (1-blue_exp_res) * win_mul
        red_delta = base_value * (0-red_exp_res) * loss_mul
    else:
        blue_delta = base_value * (0-blue_exp_res) * loss_mul
        red_delta = base_value * (1-red_exp_res) * win_mul

    print(f'{blue_elo} {red_elo} {winner} | {blue_delta} {blue_exp_res} | {red_delta} {red_exp_res} ')
    return blue_delta, red_delta


In [14]:
for match in mh_df.itertuples():
    blue_delta, red_delta = elo_change(elo_df.at[match.blue_team_id, 'elo'], 
                           elo_df.at[match.red_team_id, 'elo'],
                           match.winning_side,
                           match.region,
                           match.stage,
                           match.event_time[:4])
    if match.blue_side == 'TSM' or  match.red_side == 'TSM':
        print(f"{match.blue_side} vs {match.red_side} at {match.tournament} \n \n")
    elo_df.at[match.blue_team_id, 'elo'] += blue_delta
    elo_df.at[match.red_team_id, 'elo'] += red_delta

1250 1250 100 | 1.25 0.5 | -1.25 0.5 
1250 1250 100 | 1.25 0.5 | -1.25 0.5 
1250 1250 100 | 1.25 0.5 | -1.25 0.5 
1250 1250 100 | 1.25 0.5 | -1.25 0.5 
1250 1250 200 | -1.25 0.5 | 1.25 0.5 
1248 1248 200 | -1.25 0.5 | 1.25 0.5 
1251 1251 200 | -1.25 0.5 | 1.25 0.5 
1251 1248 100 | 1.2428045010629727 0.502878199574811 | -1.2428045010629727 0.4971218004251891 
1248 1248 100 | 1.25 0.5 | -1.25 0.5 
1251 1251 200 | -1.25 0.5 | 1.25 0.5 
1250 1250 100 | 1.25 0.5 | -1.25 0.5 
1250 1250 200 | -1.25 0.5 | 1.25 0.5 
1250 1250 100 | 1.25 0.5 | -1.25 0.5 
1250 1250 200 | -1.25 0.5 | 1.25 0.5 
1251 1250 100 | 1.2476014768051502 0.5009594092779399 | -1.2476014768051504 0.4990405907220602 
1250 1248 200 | -1.2547970287276888 0.5019188114910755 | 1.2547970287276888 0.4980811885089244 
TSM vs IMT at LCS  Spring Regular Season 2020 
 

1248 1251 200 | -1.2428045010629727 0.4971218004251891 | 1.2428045010629727 0.502878199574811 
1251 1248 100 | 1.2428045010629727 0.502878199574811 | -1.2428045010629727

In [15]:
final_elo = elo_df.loc[(teams)].sort_values('elo', ascending = False).reset_index(drop = True)
final_elo.index = final_elo.index.rename('Ranking')
final_elo.index = final_elo.index + 1
final_elo.to_csv('results.csv')

In [16]:
final_elo.

SyntaxError: invalid syntax (<ipython-input-16-f00a0a72bd67>, line 1)